In [192]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import csv
import os
from dotenv import load_dotenv
from datetime import datetime
import sqlite3
import pandas as pd
load_dotenv(verbose=True);

In [193]:
# Constants
DEVELOPER_KEY = os.getenv('API_KEY')
SERVICE_NAME = "youtube"
VERSION = "v3"

# Requests
MAX_RESULTS = 50

# CSV filename
DATA_FILE = f'data/%s-data.csv' %datetime.today().strftime('%Y-%m-%d')
DB = 'data/ytdata.db'

# Interested Playlists
SKIENNA_PLAYLISTS = (
    'PLOtl7M3yp-DX32N0fVIyvn7ipWKNGmwpp', # Analysis of Algorithms 2016
    'PLOtl7M3yp-DVBdLYatrltDJr56AKZ1qXo', # Data Science - Fall 2016
)

MIT_PLAYLISTS = (
    'PLE18841CABEA24090', # Structure and Interpretation
    'PLE7DDD91010BC51F8', # Linear Algebra
    'PLUl4u3cNGP61Oq3tWYp6V_F-5jb5L2iHb', #Introduction to Algorithms
    'PLUl4u3cNGP63WbdFxL8giv4yhgdMGaZNA', # Introduction to Computer Science and Programming - Fall 2016
    'PLUl4u3cNGP60ZaGv5SgpIk67YnH1WqCLI', # Learn to Build your own video game with the Unity Game Engine and MS Kinect
    'PLF83B8D8C87426E44', # Fundamentals of Biology
    'PLUl4u3cNGP619EG1wp0kT-7rDE_Az5TNd', # Introduction to Computational Thinking and Data Science
    'PLUl4u3cNGP61-9PEhRognw5vryrSEVLPr', # Quantum Physics I, 2013
    'PL4C4720A6F225E074', # Introduction to Computer Science and Programming - Fall 2008 (most popular)
    'PLUl4u3cNGP63ctJIEC1UnZ0btsphnnoHR', # MIT 18.S096 Topics in Mathematics w Applications in Finance 
)

THREE_BLUE_ONE_BROWN_PLAYLIST = (
    'PLZHQObOWTQDPD3MizzM2xVFitgF8hE_ab', # Essence of Linear Algebra
    'PLZHQObOWTQDMsr9K-rj53DwVRMYO3t5Yr', # Essence of Calculus
    'PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi', # Neural networks
    'PLZHQObOWTQDNPOjrT6KVlfJuKtYTftqH6', # Differential Equations
)

HARVARD_UNIVERSITY_PLAYLIST = (
    'PL2SOU6wwxB0v1kQTpqpuu5kEJo2i-iUyf', # Algorithms for Big Data
    'PL2SOU6wwxB0uP4rJgf5ayhHWgw7akUWSf', # Advanced Algorithms (COMPSCI 224) (popular)
)

COURSEA_PLAYLIST = (
    'PLVext98k2evjIFqVggHfvecnFu4tTJK_o', # GCP Fundamentals - Google Cloud Platform Fundamentals: Core Infrastructure
    'PLVext98k2evi8mDNRo4MwIgVgSmwM3cS8', # R Programming - Introduction to R by Johns Hopkins University
)

KHAN_ACADEMY_PLAYLIST = (
    'PLSQl0a2vh4HA50QhFIirlEZRXG4yjcoGM', # Journey into cryptography
)

FREECODECAMP_PLAYLIST = (
    'PLWKjhJtqVAbmGw5fN5BQlwuug-8bDmabi', # Introduction to Computer Science, Harvard's CS50
    'PLWKjhJtqVAbluXJKKbCIb4xd7fcRkpzoz', # Introduction to Game Development
)

OTHERS_PLAYLIST = (
    'PL6cactdCCnTLkQah9GKzsJmiLbegy4dEk', # Udemy Ultimate Web Development Tutorial
    'PLC3y8-rFHvwgg3vaYJgHGnModB54rxOk3', # Codevolution - Introduction to React
)

list_of_playlists = SKIENNA_PLAYLISTS + MIT_PLAYLISTS + \
                    THREE_BLUE_ONE_BROWN_PLAYLIST + HARVARD_UNIVERSITY_PLAYLIST + \
                    COURSEA_PLAYLIST + KHAN_ACADEMY_PLAYLIST + \
                    FREECODECAMP_PLAYLIST + OTHERS_PLAYLIST

In [194]:
connection = sqlite3.connect(DB)

In [195]:
service = build(SERVICE_NAME, VERSION, developerKey=DEVELOPER_KEY)

# Defining the fetch functions
The free tier Youtube API has a maximum quota of 10,000 requests per day. When defining these fetch functions, our goal here is to reduce the quota costs.

The combine cost for fetching a video's `snippet`, `contentDetails`, and `statistics` is 7 units. This means we can at most fetch 1,428 videos per day.

The combine cost for fetching `playlistItem`'s and `playlist`'s `snippet` is 3 units. This means we can fetch 3,333 `playlistItem` or `playlist` per day.

We can minimise this cost by only fetch the statistics, and check if there are any major shifts in the view count (we may set an arbitrary tolerance at around **5%**).

We can reduce the quota cost by first checking the static parts of our data. For playlist details, this means checking whether the specific playlist is already inside the database.

## Utility Functions


In [201]:
current_playlists = pd.read_sql_query('SELECT id FROM Playlist', connection)['id'].tolist()

In [204]:
current_videos = pd.read_sql_query('SELECT id FROM "2020-05-14"', connection)['id'].tolist()

In [208]:
'A2bFN3MyNDA' in current_videos

True

In [202]:
# NO SET OPTIMISATION NEEDED HERE BRRRR!
def playlist_exists(playlist_id):
    return playlist_id in current_playlists

In [165]:
def pct_change(old, new):
    return float(((new - old) * 100)/old)

In [182]:
def test_func(video_id):
    new_stats = get_video_data(video_id, 'statistics', 'items(statistics)')
    
    if video_id in current_videos:
        new_viewCount = int(new_stats['items'][0]['statistics']['viewCount'])
        old_viewCount = pd.read_sql_query(f'SELECT views from "2020-05-14" WHERE id="%s"' %video_id, connection)['views'][0]

        if pct_change(old_viewCount, new_viewCount) < 5:
            return
        else:
            ## update cell
            print(new_stats)
    else:
        pass

In [172]:
pct_change(51250, 51285)

0.06829268292682927

## Fetch Functions

In [120]:
# Recursively fetch playlist item and returns a concatenated list of playlist videos ids
def get_playlist_item(playlist_id, npt=''):
    res = service.playlistItems().list(
        part="snippet",
        fields="nextPageToken,items(snippet(resourceId(videoId)))",
        playlistId=playlist_id,
        maxResults=MAX_RESULTS,
        pageToken=npt
    ).execute()
    
    if res.get('nextPageToken') != None:
        return res['items'] + get_playlist_item(playlist_id, res['nextPageToken'])

    return res['items']

In [83]:
def get_video_statistics(video_id):
    res = service.videos().list(
      part="snippet,statistics,contentDetails",
      id=video_id,
      fields="items(id,snippet(title,categoryId,publishedAt),statistics,contentDetails(duration))",
      maxResults=MAX_RESULTS
    ).execute()
    return res

In [126]:
# General wrapper function
def get_video_data(video_id, parts='snippet', fields='items(snippet)'):
    res = service.videos().list(
      part="snippet,statistics,contentDetails",
      id=video_id,
      fields=fields,
      maxResults=MAX_RESULTS
    ).execute()
    return res

In [85]:
def get_playlist_details(playlist_id):
    res = service.playlists().list(
        part="snippet,contentDetails",
        fields="items(snippet(channelId, title, channelTitle, description), contentDetails)",
        id=playlist_id,
        maxResults=MAX_RESULTS
    ).execute()
    return res

In [86]:
def extract_video_id(data):
    return tuple(i['snippet']['resourceId']['videoId'] for i in data)

In [116]:
def get_playlist_videos_data(playlist_id):
    with open(DATA_FILE, 'a+', newline='\n', encoding='utf-8') as csvfile:
        fieldnames = ['id', 'title', 'categoryId', 'playlistId', 'duration', 'views', 'uploadedDate']
        writer = csv.DictWriter(csvfile, dialect='excel', fieldnames=fieldnames, delimiter=',')
        fileEmpty = os.stat(DATA_FILE).st_size == 0

        if fileEmpty:
            writer.writeheader()

        playlist = get_playlist_item(playlist_id)
        playlist_videos = extract_video_id(playlist)

        for i in range(len(playlist_videos)):
            data = get_video_statistics(playlist_videos[i])['items'][0]
            writer.writerow({
                'id': data['id'],
                'title': data['snippet']['title'],
                'categoryId': data['snippet']['categoryId'],
                'playlistId': playlist_id,
                'duration': data['contentDetails']['duration'],
                'views': data['statistics']['viewCount'],
                'uploadedDate': data['snippet']['publishedAt']
            })

In [184]:
def get_playlist_metadata(playlist_id):
    with open('data/playlist-details.csv', 'a+', newline='\n', encoding='utf-8') as csvfile:
        fieldnames = ['id', 'title', 'description', 'channelTitle', 'itemCount']
        writer = csv.DictWriter(csvfile, dialect='excel', fieldnames=fieldnames, delimiter=',')
        
        fileEmpty = os.stat('data/playlist-details.csv').st_size == 0
        playlist_details = get_playlist_details(playlist_id)['items'][0]
        
        if fileEmpty:
            writer.writeheader()
            
        writer.writerow({
            'id': playlist_id,
            'title': playlist_details['snippet']['title'],
            'description': playlist_details['snippet']['description'],
            'channelTitle': playlist_details['snippet']['channelTitle'],
            'itemCount': playlist_details['contentDetails']['itemCount']
        })

# Fetch the data

In [185]:
for i in range(len(list_of_playlists)):
    if playlist_exists(list_of_playlists[i]):
        continue
    else:
        get_playlist_metadata(list_of_playlists[i])

In [ ]:
for i in range(len(list_of_playlists)):
    scrape_playlist_videos_data(list_of_playlists[i])